# Getting started :rocket:

Welcome to the cloth competition! In this notebook we will load and explore the data.

> :construction: This notebook is still under construction. Dataset format may still change. :construction:

Download the dataset from [here](https://ugentbe-my.sharepoint.com/:f:/g/personal/victorlouis_degusseme_ugent_be/Ej7meTLMnQRItbr7SlQ7EdUBJGelCQIcWIQ_ddBliGXiDA?e=bf5RbG) and extract it to `data/competition_dev`.

In [ ]:
from cloth_tools.dataset.format import competition_input_sample_filenames, load_competition_input_sample
from pathlib import Path

data_dir = Path("data")
dataset_dir = data_dir / "competition_dev"

In [ ]:
sample = load_competition_input_sample(dataset_dir, sample_index=0)